In [29]:
import numpy as np

import pandas as pd

import matplotlib.pyplot as plt

from sklearn.feature_selection import f_regression
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

import seaborn as sns

import scipy.stats as stats
from scipy.stats import f

import statsmodels.api as sm

from mlxtend.feature_selection import SequentialFeatureSelector

In [2]:
def f_test(X, y):
    model = LinearRegression(fit_intercept = True).fit(X_poly, y)
    score = model.score(X_poly, y)
    f_statistic = (score/(1-score))*((X_poly.shape[0]-X_poly.shape[1]-1)/X_poly.shape[1])
    df1 = X_poly.shape[1] - 1
    df2 = y.shape[0] - 1
    p_value = 1-f.cdf(f_statistic, df2, df1)
    
    print("F_statistic: "+ str(f_statistic))
    print("p_value: "+str(p_value))
    
    return f_statistic, p_value

In [3]:
def preprocess_data(data, target, poly_degree = 2):
    y = data[target]
    X = data.drop(columns=[target])
    X = pd.get_dummies(X, drop_first=True)
    
    poly = PolynomialFeatures(poly_degree)
    X_poly = poly.fit_transform(X)
    X_poly = pd.DataFrame(X_poly, columns= poly.get_feature_names(X.columns))
    X_poly = X_poly.drop(columns = ['1'])
    return y, X_poly, X

In [4]:
data = pd.read_csv('C://datasets//Boston Housing//Housing.csv', sep=',')

In [5]:
data

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished
...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,1820000,3000,2,1,1,yes,no,yes,no,no,2,no,unfurnished
541,1767150,2400,3,1,1,no,no,no,no,no,0,no,semi-furnished
542,1750000,3620,2,1,1,yes,no,no,no,no,0,no,unfurnished
543,1750000,2910,3,1,1,no,no,no,no,no,0,no,furnished


In [6]:
y, X_poly, X = preprocess_data(data, target = 'price')

In [79]:
X

,area,bedrooms,bathrooms,stories,parking,mainroad_yes,guestroom_yes,basement_yes,hotwaterheating_yes,airconditioning_yes,prefarea_yes,furnishingstatus_semi-furnished,furnishingstatus_unfurnished
0,7420,4,2,3,2,1,0,0,0,1,1,0,0
1,8960,4,4,4,3,1,0,0,0,1,0,0,0
2,9960,3,2,2,2,1,0,1,0,0,1,1,0
3,7500,4,2,2,3,1,0,1,0,1,1,0,0
4,7420,4,1,2,2,1,1,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,3000,2,1,1,2,1,0,1,0,0,0,0,1
541,2400,3,1,1,0,0,0,0,0,0,0,1,0
542,3620,2,1,1,0,1,0,0,0,0,0,0,1
543,2910,3,1,1,0,0,0,0,0,0,0,0,0


In [8]:
X_poly

,area,bedrooms,bathrooms,stories,parking,mainroad_yes,guestroom_yes,basement_yes,hotwaterheating_yes,airconditioning_yes,...,airconditioning_yes^2,airconditioning_yes prefarea_yes,airconditioning_yes furnishingstatus_semi-furnished,airconditioning_yes furnishingstatus_unfurnished,prefarea_yes^2,prefarea_yes furnishingstatus_semi-furnished,prefarea_yes furnishingstatus_unfurnished,furnishingstatus_semi-furnished^2,furnishingstatus_semi-furnished furnishingstatus_unfurnished,furnishingstatus_unfurnished^2
0,7420.0,4.0,2.0,3.0,2.0,1.0,0.0,0.0,0.0,1.0,...,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,8960.0,4.0,4.0,4.0,3.0,1.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,9960.0,3.0,2.0,2.0,2.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
3,7500.0,4.0,2.0,2.0,3.0,1.0,0.0,1.0,0.0,1.0,...,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,7420.0,4.0,1.0,2.0,2.0,1.0,1.0,1.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,3000.0,2.0,1.0,1.0,2.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
541,2400.0,3.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
542,3620.0,2.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
543,2910.0,3.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
_,_ = f_test(X, y)

F_statistic: 14.539985703631434
p_value: 1.1102230246251565e-16


In [10]:
_,_ = f_test(X_poly, y)

F_statistic: 14.539985703631434
p_value: 1.1102230246251565e-16


### The p_value and F_statistic strongly suggest that there is a relationship between the dependent and independent variables in both cases. (p_values are about 0)

In [11]:
model = LinearRegression()
feature_selector = SequentialFeatureSelector(
model,
k_features=13,
forward=False,
verbose=1,
scoring="neg_mean_squared_error",
n_jobs=10
)
sf = feature_selector.fit(X_poly,y)
X_selected = X_poly[list(sf.k_feature_names_)]

[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:    3.6s
[Parallel(n_jobs=10)]: Done  85 out of 104 | elapsed:    4.0s remaining:    0.8s
[Parallel(n_jobs=10)]: Done 104 out of 104 | elapsed:    4.1s finished
Features: 103/13[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done 103 out of 103 | elapsed:    0.4s finished
Features: 102/13[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done 102 out of 102 | elapsed:    0.4s finished
Features: 101/13[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done 101 out of 101 | elapsed:    0.4s finished
Features: 100/13[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done 100 out of 100 | elapsed:    0.4s finished
Features: 99/13[Parallel(n_jobs=10)]: Using backend L

In [13]:
list(sf.k_feature_names_)

['bathrooms',
 'hotwaterheating_yes',
 'area bathrooms',
 'area basement_yes',
 'bedrooms hotwaterheating_yes',
 'bedrooms furnishingstatus_unfurnished',
 'bathrooms stories',
 'bathrooms airconditioning_yes',
 'mainroad_yes^2',
 'mainroad_yes hotwaterheating_yes',
 'mainroad_yes prefarea_yes',
 'guestroom_yes airconditioning_yes',
 'basement_yes airconditioning_yes']

#### If we use interaction terms, then we have to include the baselines too!!!

In [14]:
X_selected = X_selected.assign(area=X_poly['area'])
X_selected = X_selected.assign(basement_yes=X_poly['basement_yes'])
X_selected = X_selected.assign(bathrooms=X_poly['bathrooms'])
X_selected = X_selected.assign(bedrooms=X_poly['bedrooms'])
X_selected = X_selected.assign(guestroom_yes=X_poly['guestroom_yes'])
X_selected = X_selected.assign(hotwaterheating_yes=X_poly['hotwaterheating_yes'])
X_selected = X_selected.assign(mainroad_yes=X_poly['mainroad_yes'])
X_selected = X_selected.assign(airconditioning_yes=X_poly['airconditioning_yes'])
X_selected = X_selected.assign(furnishingstatus_semi_furnished=X_poly['furnishingstatus_semi-furnished'])
X_selected = X_selected.assign(stories=X_poly['stories'])
X_selected = X_selected.assign(maintoad_yes=X_poly['mainroad_yes'])
X_selected = X_selected.assign(parking=X_poly['parking'])
X_selected = X_selected.assign(furnishingstatus_unfurnished=X_poly['furnishingstatus_unfurnished'])

In [15]:
X

,area,bedrooms,bathrooms,stories,parking,mainroad_yes,guestroom_yes,basement_yes,hotwaterheating_yes,airconditioning_yes,prefarea_yes,furnishingstatus_semi-furnished,furnishingstatus_unfurnished
0,7420,4,2,3,2,1,0,0,0,1,1,0,0
1,8960,4,4,4,3,1,0,0,0,1,0,0,0
2,9960,3,2,2,2,1,0,1,0,0,1,1,0
3,7500,4,2,2,3,1,0,1,0,1,1,0,0
4,7420,4,1,2,2,1,1,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,3000,2,1,1,2,1,0,1,0,0,0,0,1
541,2400,3,1,1,0,0,0,0,0,0,0,1,0
542,3620,2,1,1,0,1,0,0,0,0,0,0,1
543,2910,3,1,1,0,0,0,0,0,0,0,0,0


In [16]:
X_selected

,bathrooms,hotwaterheating_yes,area bathrooms,area basement_yes,bedrooms hotwaterheating_yes,bedrooms furnishingstatus_unfurnished,bathrooms stories,bathrooms airconditioning_yes,mainroad_yes^2,mainroad_yes hotwaterheating_yes,...,basement_yes,bedrooms,guestroom_yes,mainroad_yes,airconditioning_yes,furnishingstatus_semi_furnished,stories,maintoad_yes,parking,furnishingstatus_unfurnished
0,2.0,0.0,14840.0,0.0,0.0,0.0,6.0,2.0,1.0,0.0,...,0.0,4.0,0.0,1.0,1.0,0.0,3.0,1.0,2.0,0.0
1,4.0,0.0,35840.0,0.0,0.0,0.0,16.0,4.0,1.0,0.0,...,0.0,4.0,0.0,1.0,1.0,0.0,4.0,1.0,3.0,0.0
2,2.0,0.0,19920.0,9960.0,0.0,0.0,4.0,0.0,1.0,0.0,...,1.0,3.0,0.0,1.0,0.0,1.0,2.0,1.0,2.0,0.0
3,2.0,0.0,15000.0,7500.0,0.0,0.0,4.0,2.0,1.0,0.0,...,1.0,4.0,0.0,1.0,1.0,0.0,2.0,1.0,3.0,0.0
4,1.0,0.0,7420.0,7420.0,0.0,0.0,2.0,1.0,1.0,0.0,...,1.0,4.0,1.0,1.0,1.0,0.0,2.0,1.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,1.0,0.0,3000.0,3000.0,0.0,2.0,1.0,0.0,1.0,0.0,...,1.0,2.0,0.0,1.0,0.0,0.0,1.0,1.0,2.0,1.0
541,1.0,0.0,2400.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,3.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
542,1.0,0.0,3620.0,0.0,0.0,2.0,1.0,0.0,1.0,0.0,...,0.0,2.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0
543,1.0,0.0,2910.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


### In order to evaluate and compare the linear and polynomial regressiob models we use 10-fold cross validation method.

In [73]:
kf = KFold(n_splits=10)
kf.get_n_splits(X)
predictions = []
for train_index, test_index in kf.split(X):
#     print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    model = LinearRegression(fit_intercept=True).fit(X_train, y_train)
    pred = model.predict(X_test)
    predictions = np.append(predictions, pred)
orig_mse = np.sqrt(((predictions - y)**2).sum()/len(y))

In [74]:
orig_mse

1278806.5110376987

### Test MSE for original model is 1278806.5110376987

In [75]:
kf = KFold(n_splits=10)
kf.get_n_splits(X)
predictions = []
for train_index, test_index in kf.split(X_selected):
#     print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X_selected.iloc[train_index], X_selected.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    model = LinearRegression(fit_intercept=True).fit(X_train, y_train)
    pred = model.predict(X_test)
    predictions = np.append(predictions, pred)
poly_mse = np.sqrt(((predictions - y)**2).sum()/len(y))

In [76]:
poly_mse

1240090.4664717228

### Test MSE for polynomial model trained on the selected features is 1240090.4664717228

In [77]:
poly_mse < orig_mse

True

## As we see Polynomial regression has lower MSE!